## 2.18 字符串令牌解析 

你有一个字符串，你想将其从左到右解析为一个令牌流

In [1]:
text = 'foo = 23 + 42 * 10'

为了令牌化字符串，你不仅需要匹配模式，还需要指定模式的类型。比如，你可能想将字符串转化为下面这种序列对

In [2]:
tokens = [('NAME', 'foo'),
          ('EQ','='),
          ('NUM', '23'),
          ('PLUS','+'),
          ('NUM', '42'),
          ('TIMES', '*'),
          ('NUM', '10')]

为了执行这样的切分，第一步就是像下面这样利用命名捕获组的正则表达式来定义所有可能的令牌，包括空格 

In [3]:
import re

In [10]:
NAME = r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'
NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
TIMES = r'(?P<TIMES>\*)'
EQ = r'(?P<EQ>=)'
WS = r'(?P<WS>\s+)'

In [11]:
master_pat = re.compile('|'.join([NAME,NUM,PLUS,TIMES,EQ,WS]))

In [12]:
master_pat

re.compile(r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)|(?P<NUM>\d+)|(?P<PLUS>\+)|(?P<TIMES>\*)|(?P<EQ>=)|(?P<WS>\s+)',
re.UNICODE)

在上面的模式中 ?P<TOKENNAME> 用于给一个模式命名，供后面使用

下一步，为了令牌化，使用模式对象很少被人了解的 scanner() 方法。这种方法会创建一个 scanner 对象，在这个对象上，不断的调用match() 方法会一步步的扫描目标文本，每步一个匹配。

实际使用这种技术的时候，可以很容易的像下面这样将上述代码打包到一个生成器中：

In [29]:
from collections import namedtuple

In [31]:
def generate_tokens(pat,text):
    Token = namedtuple('Token',['type','value'])
    scanner = pat.scanner(text)
    for m in iter(scanner.match,None):
        yield Token(m.lastgroup,m.group())

In [33]:
#Example use
for tok in generate_tokens(master_pat,'foo = 42'):
    print (tok)

Token(type='NAME', value='foo')
Token(type='WS', value=' ')
Token(type='EQ', value='=')
Token(type='WS', value=' ')
Token(type='NUM', value='42')


In [35]:
help(iter)

Help on built-in function iter in module builtins:

iter(...)
    iter(iterable) -> iterator
    iter(callable, sentinel) -> iterator
    
    Get an iterator from an object.  In the first form, the argument must
    supply its own iterator, or be a sequence.
    In the second form, the callable is called until it returns the sentinel.



通常来讲，令牌化是很多高级文本解析与处理的第一步。为了使用上面的扫描方法，你需要记住重要的几点。

首先你必须确认你使用正则表达式指定了所有输入中可能出现的文本序列。如果有任何不可匹配的文本出现了，扫描就会停止。这也是为什么上面例子中必须指定空白字符令牌的原因

令牌的顺序也是有影响的。re 模块会按照指定号的顺序去做匹配，因此如果一个模式恰好是另一个更长模式的子串，那么你需要确定长模式写在前面。

关于更高阶的令牌化技术，可以查看PyParsing 或者 PLY 包